# Basic Power Calculation with GAPoT

This notebook demonstrates the basic usage of the Geometric Algebra Power Theory (GAPoT) framework for calculating power in electrical circuits.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gapot import GeometricPower, FourierToGA
from gapot.utils import generate_distorted_signal, rms, thd

plt.style.use('seaborn-v0_8-whitegrid')
print("GAPoT Framework loaded successfully!")

## 1. Simple Sinusoidal Case

Start with a simple single-frequency (sinusoidal) voltage and current.

In [ ]:
# Parameters
f1 = 50.0      # Fundamental frequency (Hz)
fs = 10000.0   # Sampling frequency (Hz)
cycles = 2     # Number of cycles to analyze

# Time vector
T = cycles / f1
t = np.linspace(0, T, int(T * fs), endpoint=False)

# Voltage and current (RMS values)
U_rms = 230.0  # V
I_rms = 10.0   # A
phi = np.pi/6  # Phase angle (30° lagging current)

# Generate signals
u = U_rms * np.sqrt(2) * np.sin(2 * np.pi * f1 * t)
i = I_rms * np.sqrt(2) * np.sin(2 * np.pi * f1 * t - phi)

print(f"Voltage RMS: {rms(u):.2f} V")
print(f"Current RMS: {rms(i):.2f} A")

In [ ]:
# Calculate geometric power
gp = GeometricPower(u, i, f1=f1, fs=fs)

# Display results
print("=" * 50)
print("GEOMETRIC POWER ANALYSIS")
print("=" * 50)
print(f"Active Power P      = {gp.P:10.2f} W")
print(f"Reactive Power ||M_Q|| = {gp.M_Q_norm:10.2f} var")
print(f"Distortion Power ||M_D|| = {gp.M_D_norm:10.2f} va")
print(f"Apparent Power ||M|| = {gp.S:10.2f} VA")
print(f"Power Factor PF     = {gp.PF:10.4f}")
print("=" * 50)

# Expected values (theoretical)
P_expected = U_rms * I_rms * np.cos(phi)
Q_expected = U_rms * I_rms * np.sin(phi)
print(f"\nTheoretical P = {P_expected:.2f} W")
print(f"Theoretical Q = {Q_expected:.2f} var")
print(f"Theoretical PF = {np.cos(phi):.4f}")

## 2. Visualizing the Signals

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

t_ms = t * 1000

# Voltage
axes[0].plot(t_ms, u, 'b-', linewidth=1.5, label='Voltage')
axes[0].set_ylabel('Voltage (V)')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Current
axes[1].plot(t_ms, i, 'r-', linewidth=1.5, label='Current')
axes[1].set_ylabel('Current (A)')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Instantaneous power
p_inst = u * i
axes[2].plot(t_ms, p_inst, 'g-', linewidth=1.5, label='Instantaneous Power')
axes[2].axhline(y=gp.P, color='k', linestyle='--', label=f'Average P = {gp.P:.0f} W')
axes[2].set_ylabel('Power (W)')
axes[2].set_xlabel('Time (ms)')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Adding Harmonics

Now let's see what happens when we add harmonic distortion.

In [ ]:
# Generate distorted signals
harmonics = [
    (5, 0.05, 0.20, 0.3),   # 5th: 5% voltage, 20% current
    (7, 0.03, 0.14, 0.2),   # 7th: 3% voltage, 14% current
]

u_dist, i_dist, t = generate_distorted_signal(
    U1=230, I1=10, phi1=phi,
    harmonics=harmonics,
    f1=f1, fs=fs, cycles=cycles
)

# Calculate THD
thd_u = thd(u_dist, f1, fs) * 100
thd_i = thd(i_dist, f1, fs) * 100

print(f"Voltage THD: {thd_u:.2f}%")
print(f"Current THD: {thd_i:.2f}%")

In [ ]:
# Analyze distorted signals
gp_dist = GeometricPower(u_dist, i_dist, f1=f1, fs=fs)

print(gp_dist.summary())

In [ ]:
# Compare sinusoidal vs distorted
comparison = {
    'Parameter': ['P (W)', '||M_Q|| (var)', '||M_D|| (va)', '||M|| (VA)', 'PF'],
    'Sinusoidal': [gp.P, gp.M_Q_norm, gp.M_D_norm, gp.S, gp.PF],
    'Distorted': [gp_dist.P, gp_dist.M_Q_norm, gp_dist.M_D_norm, gp_dist.S, gp_dist.PF]
}

print("\n" + "=" * 60)
print(f"{'Parameter':<15} {'Sinusoidal':>15} {'Distorted':>15} {'Δ (%)':>12}")
print("=" * 60)
for param, sin_val, dist_val in zip(comparison['Parameter'], 
                                      comparison['Sinusoidal'], 
                                      comparison['Distorted']):
    if sin_val != 0:
        delta = (dist_val - sin_val) / sin_val * 100
    else:
        delta = float('inf') if dist_val != 0 else 0
    print(f"{param:<15} {sin_val:>15.4f} {dist_val:>15.4f} {delta:>11.2f}%")
print("=" * 60)

## 4. Key Observations

1. **Active Power (P)**: Slightly affected by harmonics due to cross-frequency interactions
2. **Reactive Power (M_Q)**: Represents same-frequency reactive components
3. **Distortion Power (M_D)**: Non-zero only when harmonics are present - this is a unique feature of GAPoT
4. **Power Factor**: Decreases due to the presence of harmonic distortion